# Лабораторная работа №3

## Задание

Создать сеть на базе LSTM используя TensorFlow (Keras). Сеть должна принимать на вход текстовый файл и на его базе генерировать свою абракадабру. Отчет должен содержать кроме кода, обучающий файл и результат генерации.

## Реализация

### Библиотеки

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras import models, layers

2025-12-24 00:44:53.837953: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
/home/owner/.python_env/local/lib/python3.13/site-packages/keras/src/export/tf2onnx_lib.py:8: FutureWarning: In the future `np.object` will be defined as the corresponding NumPy scalar.
  if not hasattr(np, "object"):


### Датасет

В качестве датасета возьмём произведение Моби Дик

In [2]:
file = open("moby_dick.txt", mode="r", encoding="utf-8-sig")  # sig нужен чтобы убрать кодировку в начале текста
text = file.read().replace("\n", " ")  # убираем лишние переносы и преобразуем всё в одну строку
file.close()

### Токенезация (векторизация) слов

LSTM работает только с числами. Чтобы она смогла работать с текстом, слова в нём нужно токенезировать.

Для этого в Keras уже есть слой TextVectorization. Через него уберём пунктуацию, поделим слова по пробелам и выведем всё в виде целых чисел

In [3]:
vectoriser = layers.TextVectorization(standardize='lower_and_strip_punctuation', split='whitespace', output_mode='int')
vectoriser.adapt(text)
max_tokens = len(vectoriser.get_vocabulary())

I0000 00:00:1766529896.669046  150738 gpu_device.cc:2020] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 1059 MB memory:  -> device: 0, name: NVIDIA GeForce RTX 3050 Laptop GPU, pci bus id: 0000:01:00.0, compute capability: 8.6


### Модель

In [4]:
embedding_dim = 128  # embedding переводит вывод моего токенезатора в плотный вектор признаков, его размерность будет max_tokens x embedding_dim
lstm_units = 256  # параметр показывает, как много признаков запоминает модель и как хорошо улавливает контекст

model = models.Sequential([
    layers.Embedding(input_dim=max_tokens, output_dim=embedding_dim, mask_zero=True),  # mask_zero чтобы все слои поддерживали masking
    layers.LSTM(units=lstm_units),
    layers.Dense(units=max_tokens, activation="softmax")  # именно он и будет предсказывать следующий токен
])

In [5]:
model.compile(loss='sparse_categorical_crossentropy', optimizer='adam')  # именно такая функция потерь, т.к. здесь много классов

### Скользящее окно для обучающих данных

In [6]:
vectorised_text = vectoriser(text)

X, y = [], []
window_size = 20  # сколько слов модель запоминает, чтобы предказать следующее

for i in range(0, len(vectorised_text) - window_size, window_size):
    X.append(vectorised_text[i:i+window_size])
    y.append(vectorised_text[i+window_size])

X = tf.stack(X)
y = tf.stack(y)

### Обучение модели

In [7]:
model.fit(X, y, epochs=50)

Epoch 1/50


2025-12-24 00:45:05.277713: I external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:473] Loaded cuDNN version 91002


326/326 ━━━━━━━━━━━━━━━━━━━━ 6s 11ms/step - loss: 7.6810
Epoch 2/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 6.6365
Epoch 3/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 6.3760
Epoch 4/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 6.1243
Epoch 5/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 5.8963
Epoch 6/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 5.6791
Epoch 7/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 5.4662
Epoch 8/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 5.2723
Epoch 9/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 5.0750
Epoch 10/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step - loss: 4.9116
Epoch 11/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 4.6547
Epoch 12/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 4.4084
Epoch 13/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 4.1849
Epoch 14/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step - loss: 3.9372
Epoch 15/50
326/326 ━━━━━━━━━━━━━━━━━━━━ 4s 11ms/step 

### Генерация слов

In [8]:
def sample(probabilities, temperature=0.8):  # это нам поможет брать не самое вероятное слово на каждом шаге, а слегка другие, чтобы вывод не зацикливался
    probabilities = np.asarray(probabilities).astype("float64")
    probabilities = probabilities[2:]  # 2 здесь нужна, т.к. первые два индекса в словаре не несут значения

    probabilities = np.log(probabilities + 1e-9) / temperature  # 10^-9 было добавлено во избежание 0 в логарифме

    exp_probs = np.exp(probabilities)  # возвели в экспоненту, тем самым перешли от логарифма опять к вероятностям
    probabilities = exp_probs / np.sum(exp_probs)

    return np.random.choice(max_tokens-2, p=probabilities)+2  # вернули 2 чтобы не выбрать случайно первые 2 индекса

In [9]:
def generate_text(model, vectoriser, input_text, num_words):
    tokens = vectoriser([input_text])
    output_text = input_text

    for i in range(num_words):
        preds = model.predict(tokens, verbose=0)  # выведет вероятность каждого слова
        next_id = sample(preds[0])  # выбирает индекс слова исходя из вероятности и небольшой случайности, определяемой температурой
        next_word = vectoriser.get_vocabulary()[next_id]  # возвращает слово по индексу

        output_text += " " + next_word

    return output_text

In [10]:
print(generate_text(model, vectoriser, "what", 100))

what noiseless northward scuttle chin american unlacing fast masterless leeward embalmed versions charge fool snowshoes harpooneers noiseless rude green porch seethings scuttle fashioned added terrific period nursing lean grimness vicinity street rot death stroll work felonious forefinger final merchant sun vicinity extreme rosewater sheetiron cleansed remaining oar pursuit disjointedly floorscrewed howl paddling wreck czar whose splash flume pipe binnacle hidden pyramid mother folio ivory—oh snowline lookout bison sudden dimly squall elasticity potatoes “shark” mind log justly period noiseless faraway nantucket transferred chimney scuttle cities afflictions vicinity stake horizontal grim land elbowed scuttle food reckless tragic starry bitterest mind hurried certain carpenter
